### Import libs

In [2]:
import pandas as pd

### Load data

In [33]:
pd_dataset = pd.read_csv("./data/ds_salaries.csv");
pd_dataset.head(10)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
5,2023,SE,FT,Applied Scientist,222200,USD,222200,US,0,US,L
6,2023,SE,FT,Applied Scientist,136000,USD,136000,US,0,US,L
7,2023,SE,FT,Data Scientist,219000,USD,219000,CA,0,CA,M
8,2023,SE,FT,Data Scientist,141000,USD,141000,CA,0,CA,M
9,2023,SE,FT,Data Scientist,147100,USD,147100,US,0,US,M


### Saving csv on modification

In [36]:
def save_csv(dataframe, file_name, append=False):
    mode = 'a' if append else 'w'
    dataframe.to_csv(f"./data/{file_name}", mode=mode, index=False)

### Convert location code to geo location using `geopy`, `pycountry`

In [35]:
from geopy.geocoders import Nominatim
import pycountry
from geopy.exc import GeocoderTimedOut


def get_coordinates(country_code):
    try:
        country_obj = pycountry.countries.get(alpha_2=country_code)
        if country_obj:
            geolocator = Nominatim(user_agent="geo_locator")
            location = geolocator.geocode(country_obj.name, timeout=1000000)
            return location.latitude, location.longitude
    except (AttributeError, GeocoderTimedOut):
        return None, None

pd_dataset_1 = pd_dataset_1.iloc[0:100]


# Apply the function to the specified columns
pd_dataset_1[['employee_residence_Latitude', 'employee_residence_Longitude']] = pd_dataset_1['employee_residence'].apply(lambda x: pd.Series(get_coordinates(x) if x else (None, None)))
pd_dataset_1[['company_location_Latitude', 'company_location_Longitude']] = pd_dataset_1['company_location'].apply(lambda x: pd.Series(get_coordinates(x) if x else (None, None)))

#save the data
save_csv(pd_dataset_1, "salaries_with_geo.csv")

GeocoderServiceError: 10054